# Using Twitter API to Pull Tweets

This notebook provides the workflow for pulling tweets that are then used in building the political candidate money flow knowledge graph

In [16]:
import pandas as pd
import numpy as np
import ast

In [5]:
# Load candidate twitter usernames

cand_twitters = pd.read_csv("candidate_twitters.csv")

In [6]:
cand_twitters.head()

,Name,Twitter_username,Account_start_time,Account_ID,Sex,Birthplace,Birthday,Age,Instagram_username,Political_party
0,A. Donald McEachin,RepMcEachin,2017-01-03T00:00:00Z,816181091673448448,male,Germany,1961-10-10T00:00:00Z,59.0,repmceachin,Democratic Party
1,Aaron Michlewitz,RepMichlewitz,2010-06-27T00:00:00Z,160246973,male,United States of America,1978-01-01T00:00:00Z,42.0,NaN,Democratic Party
2,Aaron Peskin,AaronPeskin,2010-11-13T00:00:00Z,215369273,male,United States of America,1964-06-17T00:00:00Z,56.0,apeskin52,Democratic Party
3,Aaron Peña,AaronPena,2007-10-31T00:00:00Z,9843332,male,United States of America,1959-06-08T00:00:00Z,61.0,NaN,Republican Party
4,Aaron Schock,aaronschock,2009-03-12T00:00:00Z,23951197,male,United States of America,1981-05-28T00:00:00Z,39.0,aaronschock,Republican Party


In [2]:
# This will import the Twarc2 client and expansions class from twarc library and also the json library
from twarc import Twarc2, expansions
import json

# This is where you initialize the client with your own bearer token (replace the XXXXX with your own bearer token)
client = Twarc2(bearer_token="AAAAAAAAAAAAAAAAAAAAAH8MiwEAAAAAhxBwj2PzTLJqURrByB7nteUOgoY%3DQ8EREzO3zgq2WQOrQu41PZ9nE48iEDCDSmHzfzyVl7Cx8jIqyk")

### Load recent tweets from a particular candidate

In [74]:
def get_tweet():
    # This timeline functions gets the Tweet timeline for a specified user
    user_timeline = client.timeline(user="SpeakerPelosi")

    # Twarc returns all Tweets for the criteria set above, so we page through the results
    for page in user_timeline:
        # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
        # so we use expansions.flatten to get all the information in a single JSON
        result = expansions.flatten(page)
        for tweet in result:
            # Here we are printing the full Tweet object JSON to the console
            tweet_json = tweet
            break
        break
    return(tweet_json)

if __name__ == "__main__":
    t = get_tweet()

In [75]:
t

{'context_annotations': [{'domain': {'id': '10',
    'name': 'Person',
    'description': 'Named people in the world like Nelson Mandela'},
   'entity': {'id': '940705992290975744',
    'name': 'Nancy Pelosi',
    'description': 'US House of Representatives Minority Leader Nancy Pelosi (CA-12)'}},
  {'domain': {'id': '35',
    'name': 'Politician',
    'description': 'Politicians in the world, like Joe Biden'},
   'entity': {'id': '940705992290975744',
    'name': 'Nancy Pelosi',
    'description': 'US House of Representatives Minority Leader Nancy Pelosi (CA-12)'}},
  {'domain': {'id': '131',
    'name': 'Unified Twitter Taxonomy',
    'description': 'A taxonomy of user interests. '},
   'entity': {'id': '847878884917886977',
    'name': 'Politics',
    'description': 'Politics'}},
  {'domain': {'id': '131',
    'name': 'Unified Twitter Taxonomy',
    'description': 'A taxonomy of user interests. '},
   'entity': {'id': '940705992290975744',
    'name': 'Nancy Pelosi',
    'descriptio